<a href="https://colab.research.google.com/github/jayasrisng/Geospatial-Data/blob/main/Geospatial_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'user-tracking-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4968641%2F8360460%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240508%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240508T223542Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1853b505b586594949d282a97c9a2d6e1e09af220cac265008cd524f2d8cb0ccfb1c4565fab3d6155d11c7b5b7ad696febe28768a6ef63742a54863d8d195e9f4a8449fcc224afcea20626cd4b20de2c69558181babeddce5f6fe1e67b4a9e1c4706dc43b90c6bb3279ecbde5a7b125e8741f50dee2f12372750f291f6ede3ff9a2851cb0c62537bac7a56f67fa99b80db89b857ced33b1def584bcb7ae52bc57d1d7e430fca33116311c633bdde806f52ff49d5d93b0f933f68625297253f2f6c986734da2e2d96bbfa7c8631042beea58ef51f8daa271f7b8c3cc00d1b5ae7486145c158f4e1b1589f65f6f56bb3c8cb852a7c352174d55f086af7454dc819'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


Preparing Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data_path = '/kaggle/input/user-tracking-dataset/combined_data.csv'
data = pd.read_csv(data_path)

# List of numeric feature columns, excluding 'Level Name' to exclude the non-numeric columns from your scaling process
feature_columns = [
    'Elapsed Time', 'Player Body Position X', 'Player Body Position Y',
    'Player Body Position Z', 'Player Body Rotation X', 'Player Body Rotation Y',
    'Player Body Rotation Z', 'Player Body Rotation W', 'Player Head Rotation X',
    'Player Head Rotation Y', 'Player Head Rotation Z', 'Player Head Rotation W'
]

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(data[feature_columns])
scaled_data = np.array(scaled_data, dtype=np.float32)

# Batch data
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices(scaled_data).shuffle(len(scaled_data)).batch(batch_size)

Define the GAN Architecture

In [ ]:
import numpy as np
def make_generator_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(256, use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(np.prod([input_dim]), activation='tanh'),  # Adjust the output size as needed
    ])
    return model

def make_discriminator_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1),
    ])
    return model

generator = make_generator_model(len(feature_columns))
discriminator = make_discriminator_model(len(feature_columns))

Define Loss Functions and Optimizers

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

Training Loop

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, len(feature_columns)])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(f'Epoch {epoch+1} Completed')

In [ ]:
epochs = 50
train(train_dataset, epochs)

Epoch 1 Completed
Epoch 2 Completed
Epoch 3 Completed
Epoch 4 Completed
Epoch 5 Completed
Epoch 6 Completed
Epoch 7 Completed
Epoch 8 Completed
Epoch 9 Completed
Epoch 10 Completed
Epoch 11 Completed
Epoch 12 Completed
Epoch 13 Completed
Epoch 14 Completed
Epoch 15 Completed
Epoch 16 Completed
Epoch 17 Completed
Epoch 18 Completed
Epoch 19 Completed
Epoch 20 Completed
Epoch 21 Completed
Epoch 22 Completed
Epoch 23 Completed
Epoch 24 Completed
Epoch 25 Completed
Epoch 26 Completed
Epoch 27 Completed
Epoch 28 Completed
Epoch 29 Completed
Epoch 30 Completed
Epoch 31 Completed
Epoch 32 Completed
Epoch 33 Completed
Epoch 34 Completed
Epoch 35 Completed
Epoch 36 Completed
Epoch 37 Completed
Epoch 38 Completed
Epoch 39 Completed
Epoch 40 Completed
Epoch 41 Completed
Epoch 42 Completed
Epoch 43 Completed
Epoch 44 Completed
Epoch 45 Completed
Epoch 46 Completed
Epoch 47 Completed
Epoch 48 Completed
Epoch 49 Completed
Epoch 50 Completed
